In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
from math import radians

import matplotlib.pyplot as plt


import prep_plays

Acquire.py Loaded Successfully
Prep.py Loaded Successfully


Can we figure out the distance between the nearest defender and the intended reciever?
- Steps to go about solving this problem
    - merging play.csv to week{number}.csv
        - play id is only unique to game id, can concat both numbers to create a unique number
    - using play description to find the intended receiver
        - will be the second name in the list of names
        - given as first initial and last name
    - extract the name
    - match the name given the play id
        - the week{number}.csv gives both first and last name
    - out of all players involved in the play, who is the closest defender

# Creating a function for Plays and Weeks CSV

## Function for combining week(number).csv and plays data

In [2]:
#this function filters out the data to where the only observation
#will be the 'pass_forward' event
prep_plays.filter_nfl_weeks()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [3]:
#this function creates the shortest distance from the receiver 
#along with additional information
df = prep_plays.combine_all_weeks_and_plays()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [4]:
df

,playDescription,quarter,down,yardsToGo,team_by_comp_yds,defendersInTheBox,numberOfPassRushers,QB_under_pressure,gameClock,absoluteYardlineNumber,...,four_three,three_four,nickel,dime,playid,closest_dist,closest_x,closest_y,defender_receiver,week
0,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,0,0,1,0,201809060075,4.16005,76.53,44.93,Jalen Mills,1
1,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,3,7.0,4.0,0.0,13:10:00,49.0,...,0,0,1,0,2018090600146,4.24323,46.18,41.1,Malcolm Jenkins,1
2,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,3,6.0,4.0,0.0,13:05:00,49.0,...,0,0,1,0,2018090600168,5.33835,46.59,12.67,Sidney Jones,1
3,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,3,6.0,5.0,1.0,13:01:00,49.0,...,0,0,0,1,2018090600190,4.24322,39.77,22.27,Malcolm Jenkins,1
4,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,3,8.0,6.0,0.0,10:59:00,11.0,...,0,0,0,0,2018090600256,2.12438,8.2,33.31,Nate Gerry,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16820,(2:19) (Shotgun) P.Rivers pass short right to ...,4,2,2,19,6.0,4.0,0.0,02:19:00,77.0,...,0,0,1,0,20181222013842,4.7633,70.56,36.45,Brandon Carr,16
16821,(2:00) (Shotgun) P.Rivers pass short left to K...,4,1,10,19,4.0,5.0,0.0,02:00:00,70.0,...,0,0,0,1,20181222013884,2.07617,56.12,11.13,Jimmy Smith,16
16822,(1:41) (Shotgun) P.Rivers pass incomplete shor...,4,1,10,19,5.0,5.0,0.0,01:41:00,53.0,...,0,0,0,1,20181222013909,0,0,0,unknown,16
16823,(1:37) (Shotgun) P.Rivers pass incomplete deep...,4,2,10,19,4.0,4.0,0.0,01:37:00,53.0,...,0,0,0,1,20181222013931,2.74168,42.5,12.68,Brandon Carr,16


The original dataframe had a little over 17,000 rows so we lost roughly 200 data points

Some receivers are unlabeled and have created "null values"

In [6]:
# a little over 800 "null values"
# we can leave them as is or fill them in with the mean/median
# this needs to be done after we train, validate and test split
df[df.closest_dist == 0]

,playDescription,quarter,down,yardsToGo,team_by_comp_yds,defendersInTheBox,numberOfPassRushers,QB_under_pressure,gameClock,absoluteYardlineNumber,...,four_three,three_four,nickel,dime,playid,closest_dist,closest_x,closest_y,defender_receiver,week
10,(5:38) M.Ryan pass incomplete short left.,1,2,7,3,7.0,4.0,0.0,05:38:00,17.0,...,0,0,1,0,2018090600545,0,0,0,unknown,1
73,(:24) (Shotgun) M.Ryan pass incomplete short r...,4,1,10,3,6.0,4.0,1.0,00:24:00,20.0,...,0,0,0,1,20180906004366,0,0,0,unknown,1
75,(:11) (Shotgun) M.Ryan pass incomplete short r...,4,3,10,3,6.0,4.0,0.0,00:11:00,20.0,...,0,0,0,1,20180906004417,0,0,0,unknown,1
98,(10:03) (Shotgun) T.Taylor pass incomplete sho...,2,1,10,11,6.0,4.0,1.0,10:03:00,67.0,...,0,0,1,0,20180909011330,0,0,0,unknown,1
287,(8:12) B.Gabbert pass incomplete short right t...,4,2,3,29,8.0,4.0,1.0,08:12:00,60.0,...,1,0,0,0,20180909033485,0,0,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16738,(2:01) M.Mariota pass incomplete short right [...,2,2,4,29,7.0,4.0,0.0,02:01:00,52.0,...,0,0,1,0,20181222001560,0,0,0,unknown,16
16745,(11:33) B.Gabbert pass incomplete short right.,3,1,10,29,7.0,6.0,1.0,11:33:00,61.0,...,0,0,1,0,20181222002073,0,0,0,unknown,16
16791,(:55) P.Rivers pass incomplete short left.,2,2,10,19,6.0,4.0,0.0,00:55:00,90.0,...,0,0,0,1,20181222011810,0,0,0,unknown,16
16813,(10:59) (Shotgun) P.Rivers pass short left to ...,4,3,6,19,5.0,5.0,0.0,10:59:00,80.0,...,0,0,0,1,20181222013351,0,0,0,unknown,16


## checking null values from combine week_and_plays function

###checking null values
newdf[newdf.closest_x == 0]

**From week 1, these plays have no information on them because of the following**
these plays have no intended receiver(15)
- 2018090600545	
- 20180906004366
- 20180906004417
- 20180909011330
- 20180909003607
- 20180909003702
- 20180909073055
- 20180909063808
- 2018090909249
- 2018090910477
- 20180909122115
- 20180909122519	
- 2018091000175
- 20180910012575	
- 20180910013099

these plays mislabeled the intended receiver
- 20180909033485
- 20180909034028
- 20180909034072
    - J.Smith(3)
    - this was changed to J.Smith-Schuster because he gets more reception and is less of a loss

- 20180909072787
- 20180909073716
    - J.O'Shaughnessy(2)
    - labeled as None

###checking plays above
xyz = df[df.playid == '20180909073716']

###obtaining play description
#must insert correct quarterback
xyz.loc[xyz.displayName == "Tom Brady"].T

###comparing names of last two rows shown
xyz.T

# Football Field Demo

## Library and Function for Field

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly

In [ ]:
def create_animation(week_number,play_id):
    df = pd.read_csv('week'+ str(week_number) +'.csv')
    dd=df[(df.playId==play_id)]
    times = dd.time.unique()
    time_slice = times[0:500:5]
    df = dd[dd.time.isin(time_slice)]
                     
    colorsIdx = {'home': 'red', 'away': 'blue', 'football': 'orange'}
    fig = px.scatter(df, x='x', y='y', color='team', text='jerseyNumber', animation_frame='time', 
                     color_discrete_map=colorsIdx,
                     range_x=[-10, 110],range_y=[-10, 53.3],
                     hover_data=['displayName', 'position', 's', 'a', 'dis', 'o', 'playDirection']
                    )
    fig.update_traces(marker_size=20)
    
    fig.update_layout(paper_bgcolor='#29a500', plot_bgcolor='#29a500', font_color='white',
        width = 800,
        height = 600,
        title = "",
        
        xaxis = dict(
        nticks = 10,
        title = "",
        visible=False
        ),
        
        yaxis = dict(
        scaleanchor = "x",
        title = "Temp",
        visible=False
        ),
        showlegend= True,
  
        annotations=[
       dict(
            x=-5,
            y=12,
            xref="x",
            yref="y",
            text="HOME ENDZONE",
            font=dict(size=16,color="#e9ece7"),
            align='center',
            showarrow=False,
            yanchor='bottom',
            textangle=-90
        ),
        dict(
            x=105,
            y=10,
            xref="x",
            yref="y",
            text="VISITOR ENDZONE",
            font=dict(size=16,color="#e9ece7"),
            align='center',
            showarrow=False,
            yanchor='bottom',
            textangle=90
        )]  
        ,
        legend=dict(
        traceorder="normal",
        font=dict(family="sans-serif",size=12),
        title = "",
        orientation="h",
        yanchor="bottom",
        y=1.00,
        xanchor="center",
        x=0.5
        ),
    )
    ####################################################
        
    fig.add_shape(type="rect", x0=-10, x1=0,  y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#217b00" ,layer="below")
    fig.add_shape(type="rect", x0=100, x1=110, y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#217b00" ,layer="below")
    for x in range(0, 100, 10):
        fig.add_shape(type="rect", x0=x,   x1=x+10, y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#29a500" ,layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=1, x1=x, y1=2,line=dict(color="#c8ddc0",width=2),layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=51.3, x1=x, y1=52.3,line=dict(color="#c8ddc0",width=2),layer="below")
    
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=20.0, x1=x, y1=21,line=dict(color="#c8ddc0",width=2),layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=32.3, x1=x, y1=33.3,line=dict(color="#c8ddc0",width=2),layer="below")
    
    
    fig.add_trace(go.Scatter(
    x=[2,10,20,30,40,50,60,70,80,90,98], y=[5,5,5,5,5,5,5,5,5,5,5],
    text=["G","1 0","2 0","3 0","4 0","5 0","4 0","3 0","2 0","1 0","G"],
    mode="text",
    textfont=dict(size=20,family="Arail"),
    showlegend=False,
    ))
    
    fig.add_trace(go.Scatter(
    x=[2,10,20,30,40,50,60,70,80,90,98], y=[48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3],
    text=["G","1 0","2 0","3 0","4 0","5 0","4 0","3 0","2 0","1 0","G"],
    mode="text",
    textfont=dict(size=20,family="Arail"),
    showlegend=False,
    ))
    
    return fig

## Function test

In [ ]:
week1 = pd.read_csv('week1.csv')

In [ ]:
[play for play in week1.playId.unique()]

## Animation

In [ ]:
create_animation(1,75)

In [ ]:
week1 = pd.read_csv('week1.csv')

In [ ]:
week1

In [ ]:
week1.event.value_counts()

In [ ]:
df = week1[week1.event != 'None']

In [ ]:
df

## Scratch Work Past This Point

# Prep week csv

## clean nfl weeks(drop noise)

In [ ]:
def filter_nfl_weeks():
    
    for i in range(1,18):
        #read in csv
        df = pd.read_csv('week' + str(i) + '.csv')
        #keep only pass_forward events
        df = df[(df.event == 'pass_forward')]
        #fill in empty positions with football
        df.position = df.position.fillna('BALL')
        #reset index
        df.reset_index(drop=True)
        #save as new csv
        df.to_csv('week' + str(i) + 'filtered.csv', index=False)
        #print week number when done
        print(f'{i}')

In [ ]:
filter_nfl_weeks()

## plays

In [ ]:
#read in csv
plays = prep_plays.prep_plays_for_weeks()

In [ ]:
plays.T

In [ ]:
#since playId is only unique to game id we need to combine to make a unique identifier
#first have to change the data types to strings
plays.gameId = plays.gameId.astype(str)
plays.playId = plays.playId.astype(str)

In [ ]:
#concat to create a unique identifier
plays['playid'] = plays.gameId + plays.playId

In [ ]:
#drop old columns
plays = plays.drop(columns = {'gameId', 'playId'})

In [ ]:
plays.drop_duplicates(inplace=True)

## week1

In [ ]:
#read in csv
wk1 = pd.read_csv('week10filtered.csv')

In [ ]:
wk1 = wk1.drop_duplicates()

In [ ]:
#since playId is only unique to game id we need to combine to make a unique identifier
#first have to change the data types to strings
wk1.gameId = wk1.gameId.astype(str)

wk1.playId = wk1.playId.astype(str)

In [ ]:
#concat to create a unique identifier
wk1['playid'] = wk1.gameId + wk1.playId

In [ ]:
wk1 = wk1.drop(columns = {'gameId', 'playId'})

In [ ]:
wk1.drop_duplicates(inplace=True)

## merge

In [ ]:
#merge 
df = pd.merge(plays, wk1, left_on = 'playid', right_on = 'playid', how = 'inner')

In [ ]:
df.drop_duplicates(inplace=True)

## Identify reciever

In [ ]:
import re

In [ ]:
#extracting names from play description
#second name will be the intended reciever
desc = df.playDescription.str.findall(r'(\b[A-Z]+\.\b[A-Z]+\w+)').apply(','.join)

In [ ]:
desc

In [ ]:
#split the desc namees by comma
temp = desc.str.split(',', expand = True)

In [ ]:
temp

In [ ]:
#checking reciever column for nulls
#there are some
#will have to write a loop that considers nulls and skips them
temp[1].isnull().value_counts()

In [ ]:
# obtaining name from WR column
temp[1]

In [ ]:
#saving name of receiver
df['receiver_last_name'] = temp[1]

In [ ]:
#checking dataframe
df

In [ ]:
#splitting first and last name of player
temp = df.displayName.str.split(' ', expand = True)

In [ ]:
temp

In [ ]:
initial = temp[0].astype(str).str[0]
initial

In [ ]:
last_name = temp[1]

In [ ]:
df['player_last_name'] = initial + '.' + last_name

In [ ]:
df = df[df.displayName != 'Football']

In [ ]:
#checking dataframe
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
df.loc[(df.receiver_last_name == 'J.Smith'),'receiver_last_name']='J.Smith-Schuster'

In [ ]:
df.loc[(df.receiver_last_name == 'A.Seferian'),'receiver_last_name']='A.Seferian-Jenkins'

In [ ]:
df.loc[(df.receiver_last_name == 'R.Seals'),'receiver_last_name']='R.Seals-Jones'

In [ ]:
df.drop_duplicates(inplace=True)

## Distance

In [ ]:
# #passes where no receiver was identified
# df = df[~df.receiver_last_name.isnull()]

In [ ]:
# #no receiver was identified
# df = df[df.playid != '20180906004417']

In [ ]:
df

In [ ]:
newdf = pd.DataFrame(columns = ['playids', 'closest_dist', 'closest_x', 'closest_y', 'defender_receiver', 'week'])
playids = [
 
 
 
 
 
 
 
 
 '20181111062650',
 
]

#loop through each playid in playids
for play in playids:
    #reset shortest distance
    closest_distance = 100
    #reset shortest x
    closest_x = 0
    #reset shortest y
    closest_y = 0
    #filter for all players in current play
    current_play = df[df.playid == play]
    #create a dataframe of offensive players
    offense = current_play[(current_play.position == 'QB') | (current_play.position == 'RB') | (current_play.position == 'WR') | (current_play.position == 'FB') | (current_play.position == 'HB') | (current_play.position == 'TE')]
    #create a dataframe of defensive players
    defense = current_play[(current_play.position == 'CB') | (current_play.position == 'OLB') | (current_play.position == 'FS') | (current_play.position == 'SS') | (current_play.position == 'ILB') | (current_play.position == 'MLB') | (current_play.position == 'LB') | (current_play.position == 'DB') | (current_play.position == 'S') | (current_play.position == 'DL') | (current_play.position == 'DE') | (current_play.position == 'NT')]
    #for x in coordinates of players
    for name in defense.displayName:
        if (offense.receiver_last_name == offense.player_last_name).any():
            #retrieve y coordinate of this player
            x = defense.loc[defense.displayName == name].x.item()
            #retrieve y coordinate of this player
            y = defense.loc[defense.displayName == name].y.item()
            #retrive x coordinate of reciever
            x1= offense.loc[offense.receiver_last_name == offense.player_last_name].x.item()
            # retrieve y coordinate of reciever
            y1= offense.loc[offense.receiver_last_name == offense.player_last_name].y.item()
            #solve for distance
            distance = ((x-x1)**2+(y-y1)**2)**(1/2)
            #if the distance is the shortest distance
            if distance < closest_distance:
                #save the distance
                closest_distance = distance
                #save the x coordinate
                closest_x = x
                #save the y coordinate
                closest_y = y
                #save the defender name
                def_name = name
        else:
            #fill with unrealistic values 
            closest_distance = 0
            closest_x = 0
            closest_y = 0
            def_name = "unknown"
    newdf = newdf.append({'playids': play, 'closest_dist': closest_distance, 'closest_x': closest_x, 'closest_y': closest_y, 'defender_receiver': def_name, 'week':1}, ignore_index=True)

In [ ]:
newdf

In [ ]:
[play for play in df.playid.unique()]

# End